In [0]:
%run ./00_Setup

In [0]:
%sql
-- Set the active schema to strata_lab
USE SCHEMA strata_lab;

In [0]:
%sql
-- Create a widget to store the book store data path
CREATE WIDGET TEXT book_store_path DEFAULT '/Volumes/workspace/strata_lab/entrenamiento/book_store';

In [0]:
%sql
-- Create the customers table from JSON data
CREATE OR REPLACE TABLE customers AS
SELECT * FROM json.`${book_store_path}/customers`;

In [0]:
%sql
-- Preview the customers table
SELECT * FROM customers

In [0]:
%sql
-- Show the schema of the customers table
DESCRIBE TABLE customers

In [0]:
%sql
-- Query specific fields from JSON profile in customers
SELECT
  customer_id,
  profile:first_name,
  profile:address:country
FROM customers

In [0]:
%sql
-- Example: Convert JSON profile to struct type (requires schema)
/*
SELECT
  from_json(profile) AS profile_struct
FROM customers
*/

In [0]:
%sql
-- Get a sample profile JSON to infer schema
SELECT
  profile
FROM customers
LIMIT 1

In [0]:
%sql
-- Parse profile JSON using schema inferred from sample data
CREATE OR REPLACE TEMP VIEW parsed_customers AS
SELECT
  customer_id,
  from_json(profile, schema_of_json('{"first_name":"Linda","last_name":"Lee","gender":"Female","address":{"street":"0723 Ramirez Terrace Apt. 096","city":"Port Brittany","country":"USA"}}')) AS profile_struct
FROM customers

In [0]:
%sql
-- Preview the parsed_customers view
SELECT * FROM parsed_customers
LIMIT 5

In [0]:
%sql
-- Show the schema of the parsed_customers view
DESCRIBE parsed_customers

In [0]:
%sql
-- For struct type, use dot instead of colon syntax

In [0]:
%sql
-- Query fields from struct profile in parsed_customers
SELECT 
  customer_id,
  profile_struct.first_name,
  profile_struct.address.country
FROM parsed_customers

In [0]:
%sql
-- Flatten struct fields into columns in customers_final view
CREATE OR REPLACE TEMP VIEW customers_final AS
SELECT
  customer_id,
  profile_struct.*
FROM parsed_customers;

-- Preview the customers_final view
SELECT * FROM customers_final

In [0]:
%sql
-- Create the orders table from Parquet data
CREATE OR REPLACE TABLE orders AS
SELECT * FROM parquet.`${book_store_path}/orders`

In [0]:
%sql
-- Preview the orders table
SELECT
  order_id,
  customer_id,
  books
FROM orders
LIMIT 5

In [0]:
%sql
-- Explode books array to flatten into new rows
SELECT
  order_id, 
  customer_id,
  explode(books) AS book
FROM orders
LIMIT 5

In [0]:
%sql
-- Collect unique order IDs and book IDs per customer
SELECT
  customer_id,
  collect_set(order_id) AS order_set,
  collect_set(books.book_id) AS books_set
FROM orders
GROUP BY customer_id
LIMIT 5

In [0]:
%sql
-- Flatten and deduplicate book IDs per customer
SELECT
  customer_id,
  collect_set(books.book_id) AS before_flatten,
  array_distinct(flatten(collect_set(books.book_id))) AS after_flatten
FROM orders
GROUP BY customer_id
LIMIT 5

In [0]:
%sql
-- Join orders with books to enrich order details
-- Note: Book price in order and books table may differ due to random values
CREATE OR REPLACE VIEW orders_enriched AS
SELECT
  *
FROM (
  SELECT 
    *,
    explode(books) AS book
    FROM orders
) AS o
INNER JOIN books b
ON o.book.book_id = b.book_id;

-- Preview the orders_enriched view
SELECT * FROM orders_enriched
LIMIT 10

In [0]:
%sql
-- Union orders with new updates from Parquet
CREATE OR REPLACE TEMP VIEW orders_updates
AS SELECT * FROM parquet.`${book_store_path}/orders_new`;

-- Show union of orders and updates
SELECT * FROM orders
UNION
SELECT * FROM orders_updates

In [0]:
%sql
-- Show intersection of orders and updates
SELECT * FROM orders
INTERSECT
SELECT * FROM orders_updates

In [0]:
%sql
-- Show orders that are not in updates
SELECT * FROM orders
MINUS
SELECT * FROM orders_updates

In [0]:
%sql
-- Pivot book quantities per customer into columns
CREATE OR REPLACE TABLE transactions AS
SELECT
  *
FROM(
  SELECT
    customer_id,
    book.book_id,
    book.quantity AS quantity
  FROM orders_enriched
) PIVOT(
sum(quantity) FOR book_id IN (
  'B01', 'B02', 'B03', 'B04', 'B05', 'B06',
  'B07', 'B08', 'B09', 'B10', 'B11', 'B12'
)
);

-- Preview the transactions table
SELECT * FROM transactions